In [1]:
import sys

sys.path.append("../")

In [2]:
import os
import time
import joblib
from joblib import Parallel, delayed
from pathlib import Path
from pprint import pprint
from typing import List, Tuple, Union, Optional

import torch 
import yaml
import faiss 
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
from sklearn import preprocessing
from matplotlib import pyplot as plt
from PIL import Image
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator, precision_at_k
from omegaconf.dictconfig import DictConfig
from torch.utils.data import DataLoader
from tqdm import tqdm

from retrieval.engine.evaluate import evaluate, get_tester
from retrieval.models.net import RetrievalNet
from retrieval.getter import Getter

data_dir = "../../data/train10k/"
DEVICE = "cuda"
WEIGHTS_PATH = Path("../experiments/ROADMAP/GLDv2_ROADMAP_128/weights/rolling.ckpt") # Path("../experiments/ROADMAP/GLDv2_ROADMAP_classification_splits_resnet_440/weights/rolling.ckpt") #
getter = Getter()
ID2CLS = {}

# Utils

In [3]:
def load_cfg(cfg_path):
    with open(cfg_path, "r") as f:
        cfg = yaml.safe_load(f)
    return DictConfig(cfg)


def recall_at_k(targets, predictions, k):
    s = sum(t in torch.Tensor(y).long()[:k] for t, y in zip(targets, predictions))
    return s / len(targets)

def global_average_precision_at_k(
    knn_labels, query_labels, knn_distances, num_k=1
):
    knn_labels = knn_labels[:, :num_k]
    knn_distances = knn_distances[:, :num_k]
    num_samples = knn_labels.shape[0]
    query_labels, knn_labels, knn_distances = (
        query_labels.flatten(),
        knn_labels.flatten(),
        knn_distances.flatten(),
    )
    _, indices = knn_distances.sort(dim=0, descending=True)
    sorted_query_labels = query_labels[indices]
    sorted_knn_labels = knn_labels[indices]
    relevance = torch.eq(sorted_query_labels, sorted_knn_labels)
    cumulative_correct = torch.cumsum(relevance, dim=0)
    precision = cumulative_correct / torch.arange(1, num_samples + 1)
    gap = (precision * relevance).sum() / num_samples
    return gap.float().item()


def embedding_inference(data_loader, model):
    model.eval()

    all_predicts, all_targets = [], []

    with torch.no_grad():
        for i, data in enumerate(tqdm(data_loader)):
            input_, target = data["image"], data["label"]
            input_ = input_.to(DEVICE)
            emb = model(input_)
            all_predicts.append(emb.squeeze().cpu())
            all_targets.append(target)

    predicts = torch.cat(all_predicts)
    targets = torch.cat(all_targets)

    return predicts, torch.tensor([ID2CLS[id_.item()] for id_ in targets.flatten()])

# Model

In [4]:
net = RetrievalNet(
    "vit_deit_distilled",
    embed_dim=384,
    norm_features=False,
    without_fc=True,
    with_autocast=True,
)
# net = RetrievalNet(
#     "resnet18",
#     embed_dim=512,
#     norm_features=False,
#     without_fc=True,
#     with_autocast=True,
# )

weights = torch.load(WEIGHTS_PATH, map_location=DEVICE)["net_state"]
net.load_state_dict(weights)
net.to(DEVICE)
net.eval() 

RetrievalNet(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU()
          (drop1): Dropout(p=0.0, inplace=False)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (d

# Data

In [5]:
transform_cfg = load_cfg("../retrieval/config/transform/gldv2.yaml")
dataset_cfg = load_cfg("../retrieval/config/dataset/gldv2_10k_classification_splits.yaml")
test_transform = getter.get_transform(transform_cfg.test)
test_ds = getter.get_dataset(test_transform, 'test', dataset_cfg)
train_ds = getter.get_dataset(test_transform, 'train', dataset_cfg)
print(len(train_ds), len(test_ds))
CLASSES = Path("./classes.txt").read_text().split() if Path("./classes.txt").exists() else train_ds.classes_
ID2CLS = {
    i: int(cls)
    for i, cls in enumerate(CLASSES)
}

78192 39482


In [6]:
train_loader = DataLoader(
    train_ds,
    batch_size=1024,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)
test_loader = DataLoader(
    test_ds,
    batch_size=1024,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)

# KNN & ANN

In [46]:
class BaseKNN(object):
    """KNN base class"""
    def __init__(self, embeddings: np.ndarray, ids: Optional[np.ndarray], method):
        if embeddings.dtype != np.float32:
            embeddings = embeddings.astype(np.float32)
        self.N = len(embeddings)
        self.D = embeddings[0].shape[-1]
        self.embeddings = embeddings if embeddings.flags['C_CONTIGUOUS'] \
                               else np.ascontiguousarray(embeddings)
        self.labels = ids

    def add(self, batch_size=10000):
        """Add data into index"""
        if self.N <= batch_size:
            self.index.add(self.embeddings)
        else:
            [self.index.add(self.embeddings[i:i+batch_size])
                    for i in range(0, len(self.embeddings), batch_size)]

    def search(self, queries, k=5):
        """Search
        Args:
            queries: query vectors
            k: get top-k results
        Returns:
            sims: similarities of k-NN
            ids: indexes of k-NN
        """
        if not queries.flags['C_CONTIGUOUS']:
            queries = np.ascontiguousarray(queries)
        if queries.dtype != np.float32:
            queries = queries.astype(np.float32)
        sims, ids = self.index.search(queries, k)
        
        ids = self.labels[ids] if self.labels else ids
        return sims, ids


class KNN(BaseKNN):
    """KNN class
    Args:
        embeddings: feature vectors in database
        ids: labels of feature vectors
        method: distance metric
    """
    def __init__(self, embeddings: np.ndarray, ids: np.ndarray, method):
        super().__init__(embeddings, ids, method)
        self.index = {
            'cosine': faiss.IndexFlatIP,
            'euclidean': faiss.IndexFlatL2,
        }[method](self.D)
        if os.environ.get('CUDA_VISIBLE_DEVICES'):
            self.index = faiss.index_cpu_to_all_gpus(self.index)
        self.labels = ids
        self.add()


class ANN(BaseKNN):
    """Approximate nearest neighbor search class
    Args:
        embeddings: feature vectors in database
        ids: labels of feature vectors
        method: distance metric
    """
    def __init__(
        self, embeddings: np.ndarray, ids: Optional[np.ndarray], method="cosine", M=128, nbits=8, nlist=316, nprobe=64
    ):
        super().__init__(embeddings, ids, method)
        self.labels = ids
        self.quantizer = {
            'cosine': faiss.IndexFlatIP,
            'euclidean': faiss.IndexFlatL2
        }[method](self.D)
        self.index = faiss.IndexIVFPQ(self.quantizer, self.D, nlist, M, nbits)
        samples = embeddings[np.random.permutation(np.arange(self.N))[:self.N // 5]]
        self.index.train(samples)
        self.add()
        self.index.nprobe = nprobe

In [5]:
train_embeddings_path = Path("./cache_embeddings/train_emb.npz")
train_targets_path = Path("./cache_embeddings/train_targets.npz")
test_embeddings_path = Path("./cache_embeddings/test_emb.npz")
test_targets_path = Path("./cache_embeddings/test_targets.npz")

if not train_embeddings_path.exists():
    train_embeddings, train_targets = embedding_inference(train_loader, net)
    train_embeddings, train_targets = train_embeddings.numpy(), train_targets.numpy()
    np.savez_compressed(train_embeddings_path, embeddigs=train_embeddings)
    np.savez_compressed(train_targets_path, embeddigs=train_targets)
else:
    train_embeddings = np.load(train_embeddings_path)["embeddigs"]
    train_targets = np.load(train_targets_path)["embeddigs"]

if not test_embeddings_path.exists():
    queries, targets = embedding_inference(test_loader, net)
    queries, targets = queries.numpy(), targets.numpy()
    np.savez_compressed(test_embeddings_path, embeddigs=queries)
    np.savez_compressed(test_targets_path, embeddigs=targets)
else:
    queries = np.load(test_embeddings_path)["embeddigs"]
    targets = np.load(test_targets_path)["embeddigs"]

In [13]:
db_cosine = KNN(train_embeddings, train_targets, method="cosine")
# db_euclidean = KNN(train_embeddings.numpy(), train_targets.numpy(), method="euclidean")
# db_ann = ANN(train_embeddings.numpy(), train_targets.numpy(), method="cosine")

[index] add: 100%|██████████| 16/16 [00:31<00:00,  1.96s/it]


In [14]:
dbs = [db_cosine]#, db_euclidean, db_ann]
names = ["Cosine"]#, "Euclidean", "ANN cosine"]
for db, name in zip(dbs, names):
    t = time.time()
    sims, ids = db.search(queries.numpy(), k=5)
    search_time = time.time() - t
    print(f"\n{name} - {search_time:.4f} secs")
    print(f"R@1 = {recall_at_k(targets, ids, k=1):.4f}")
    print(f"R@5 = {recall_at_k(targets, ids, k=5):.4f}")
    print(f"GAP@1 = {global_average_precision_at_k(torch.from_numpy(ids), targets, torch.from_numpy(sims)):.4f}")


ANN cosine
R@1 = 0.8066
R@5 = 0.8851
GAP@1 = 0.5190

Cosine
R@1 = 0.8112
R@5 = 0.8887
GAP@1 = 0.7799


In [ ]:
"""
ResNet18
R@1 = 0.8112
R@5 = 0.8887
GAP@1 = 0.7799

DeiT
R@1 = 0.9032
R@5 = 0.9507
GAP = 0.8940
"""

# Offline Diffusion

In [47]:
trunc_ids = None
trunc_init = None
lap_alpha = None


def get_offline_result(i):
    ids = trunc_ids[i]
    trunc_lap = lap_alpha[ids][:, ids]
    scores, _ = linalg.cg(trunc_lap, trunc_init, tol=1e-6, maxiter=20)
    return scores


def cache(filename):
    """Decorator to cache results"""

    def decorator(func):
        def wrapper(*args, **kw):
            self = args[0]
            path = os.path.join(self.cache_dir, filename)
            Path(self.cache_dir).mkdir(parents=True, exist_ok=True)
            time0 = time.time()
            if os.path.exists(path):
                result = joblib.load(path)
                cost = time.time() - time0
                # print("[cache] loading {} costs {:.2f}s".format(path, cost))
                return result
            result = func(*args, **kw)
            cost = time.time() - time0
            print("[cache] obtaining {} costs {:.2f}s".format(path, cost))
            joblib.dump(result, path)
            return result

        return wrapper

    return decorator


class Diffusion(object):
    """Diffusion class"""

    def __init__(self, features, labels, cache_dir):
        self.features = features
        self.labels = labels
        self.N = len(self.features)
        self.cache_dir = cache_dir
        # use ANN for large datasets
        self.use_ann = self.N >= 1_000_000
        if self.use_ann:
            print("ANN creating...")
            self.ann = ANN(self.features, self.labels, method="cosine")
        self.knn = KNN(self.features, self.labels, method="cosine")

    @cache("offline.jbl")
    def get_offline_results(self, n_trunc, kd=50):
        """Get offline diffusion results for each gallery feature"""
        print("[offline] starting offline diffusion")
        print("[offline] 1) prepare Laplacian and initial state")
        global trunc_ids, trunc_init, lap_alpha
        if self.use_ann:
            _, trunc_ids = self.ann.search(self.features, n_trunc)
            sims, ids = self.knn.search(self.features, kd)
            lap_alpha = self.get_laplacian(sims, ids)
        else:
            sims, ids = self.knn.search(self.features, n_trunc)
            trunc_ids = ids
            lap_alpha = self.get_laplacian(sims[:, :kd], ids[:, :kd])
        trunc_init = np.zeros(n_trunc)
        trunc_init[0] = 1

        print("[offline] 2) gallery-side diffusion")
        results = Parallel(n_jobs=-1, prefer="threads")(
            delayed(get_offline_result)(i)
            for i in tqdm(range(self.N), desc="[offline] diffusion")
        )
        all_scores = np.concatenate(results)

        print("[offline] 3) merge offline results")
        rows = np.repeat(np.arange(self.N), n_trunc)

        offline = sparse.csr_matrix(
            (all_scores, (rows, trunc_ids.reshape(-1))),
            shape=(self.N, self.N),
            dtype=np.float32,
        )
        return offline

    # @cache('laplacian.jbl')
    def get_laplacian(self, sims, ids, alpha=0.99):
        """Get Laplacian_alpha matrix"""
        affinity = self.get_affinity(sims, ids)
        num = affinity.shape[0]
        degrees = affinity @ np.ones(num) + 1e-12
        # mat: degree matrix ^ (-1/2)
        mat = sparse.dia_matrix(
            (degrees ** (-0.5), [0]), shape=(num, num), dtype=np.float32
        )
        stochastic = mat @ affinity @ mat
        sparse_eye = sparse.dia_matrix(
            (np.ones(num), [0]), shape=(num, num), dtype=np.float32
        )
        lap_alpha = sparse_eye - alpha * stochastic
        return lap_alpha

    # @cache('affinity.jbl')
    def get_affinity(self, sims, ids, gamma=3):
        """Create affinity matrix for the mutual kNN graph of the whole dataset
        Args:
            sims: similarities of kNN
            ids: indexes of kNN
        Returns:
            affinity: affinity matrix
        """
        num = sims.shape[0]
        sims[sims < 0] = 0  # similarity should be non-negative
        sims = sims**gamma
        # vec_ids: feature vectors' ids
        # mut_ids: mutual (reciprocal) nearest neighbors' ids
        # mut_sims: similarites between feature vectors and their mutual nearest neighbors
        vec_ids, mut_ids, mut_sims = [], [], []
        for i in range(num):
            # check reciprocity: i is in j's kNN and j is in i's kNN when i != j
            ismutual = np.isin(ids[ids[i]], i).any(axis=1)
            ismutual[0] = False
            if ismutual.any():
                vec_ids.append(i * np.ones(ismutual.sum(), dtype=int))
                mut_ids.append(ids[i, ismutual])
                mut_sims.append(sims[i, ismutual])
        vec_ids, mut_ids, mut_sims = map(np.concatenate, [vec_ids, mut_ids, mut_sims])
        affinity = sparse.csc_matrix(
            (mut_sims, (vec_ids, mut_ids)), shape=(num, num), dtype=np.float32
        )
        return affinity


def offline_diffusion_search(
    queries,
    diffusion,
    truncation_size=1000,
    kd=5,
):
    """
    Args:
        queries: predicted embeddings
        gallery: train embeddings
        cache_dir: Directory to cache embeddings
        truncation_size: Number of images in the truncated gallery
        kd: top k results
    """
    # t = time.time()
    n_query = len(queries)
    offline = diffusion.get_offline_results(truncation_size, kd)
    # nan_idx = [i for i, row in enumerate(tqdm(offline.data)) if not np.any(np.isnan(row))]
    # print(len(nan_idx))
    # print(offline.shape)
    # offline.data = offline.data[nan_idx, :]

    # print("Search NaNs")
    # cnt = 0
    # nan_indices = []
    # for i in tqdm(range(offline.shape[0])):
    #     a = offline[i].toarray()
    #     is_nans = np.any(np.isnan(a))
    #     if is_nans:
    #         cnt += 1
    #         nan_indices.append(i)
    # print(f"find {cnt} embeddings with NaNs")
    # print(f"NaN indices: {nan_indices}")

    # print("fill nans")
    # offline.data[np.isnan(offline.data)] = 0.0
    # print("normalize")
    features = preprocessing.normalize(offline, norm="l2", axis=1)
    # print("count scores")
    scores = features[:n_query] @ features[n_query:].T
    # print("sort")
    ranks = np.argsort(-scores.toarray())
    # ranks = np.argpartition(-scores.todense(), -kd)[-kd:]
    # ranks = np.argmax(scores.todense(), axis=1)
    # all_pipeline_time = time.time() - t
    # print(f"All Pipeline Time: {all_pipeline_time:.4f} seconds")
    return ranks[:, :kd]


# off_ids = offline_diffusion_search(
#     queries[:100], train_embeddings, None, truncation_size=10, kd=1
# )

In [54]:
batch_size = 4096
recalls = []
predicts = []

t = time.time()
for i in tqdm(range(0, len(queries), batch_size), desc="Diffusion prediction"):
    q = queries[i : i + batch_size]
    name = "Offline Diffusion"
    diffusion = Diffusion(
        np.vstack([q, train_embeddings]),
        None,
        cache_dir="/home/and/projects/made/landmarks-recognition/andrey-research-repo/notebooks/cache_embeddings",
    )
    ids = offline_diffusion_search(q, diffusion, truncation_size=10, kd=5)
    predicts.append(train_targets[ids])
predicts = np.concatenate(predicts)
search_time = time.time() - t

sims = np.repeat([i / 10 for i in range(9, 4)], len(queries))
print(f"\n{name} - {search_time:.4f} secs")
print(f"R@1 = {recall_at_k(targets[:batch_size], predicts, k=1):.4f}")
# print(f"R@5 = {recall_at_k(targets, predicts, k=5):.4f}")
# print(f"GAP@1 = {global_average_precision_at_k(torch.from_numpy(ids),  torch.from_numpy(targets), torch.from_numpy(sims)):.4f}")

[index] add:  38%|███▊      | 3/8 [04:10<06:57, 83.44s/it]


ValueError: Input contains NaN.

In [52]:
targets[:20]

array([ 44244, 174711,  64263,  77126,  26841,  48168, 176018,  13111,
        62440, 191756,  79505, 202154, 172056, 105193,   6669, 128357,
       202886, 107718, 126255,  55235])

In [53]:
predicts

array([[177599, 177599, 177599, 194198, 194198],
       [ 42848,  42848,  42848,  42848,  42848],
       [  7112,  74090,  74090,  74090, 137927],
       ...,
       [  7112, 121157, 121157, 121157,  21334],
       [  7112, 121157, 121157, 121157,  21334],
       [  7112, 121157, 121157, 121157,  21334]])